# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 4 2022 11:43AM,250070,10,CTF0011287-BO,"Citieffe, Inc.",Released
1,Nov 4 2022 11:43AM,250069,10,0086144657,ISDIN Corporation,Released
2,Nov 4 2022 11:43AM,250069,10,0086144658,ISDIN Corporation,Released
3,Nov 4 2022 11:43AM,250069,10,0086144660,ISDIN Corporation,Released
4,Nov 4 2022 11:41AM,250068,19,60017833,"GUSA Granules USA, Inc.",Released
5,Nov 4 2022 11:40AM,250065,10,0086143386,ISDIN Corporation,Released
6,Nov 4 2022 11:40AM,250065,10,0086143387,ISDIN Corporation,Released
7,Nov 4 2022 11:40AM,250065,10,0086143390,ISDIN Corporation,Released
8,Nov 4 2022 11:40AM,250065,10,0086143392,ISDIN Corporation,Released
9,Nov 4 2022 11:40AM,250065,10,0086143391,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,250066,Released,1
16,250067,Released,1
17,250068,Released,1
18,250069,Released,3
19,250070,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250066,NaN,1.0
250067,NaN,1.0
250068,NaN,1.0
250069,NaN,3.0
250070,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249985,0.0,1.0
250022,0.0,1.0
250034,0.0,1.0
250037,0.0,1.0
250038,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249985,0,1
250022,0,1
250034,0,1
250037,0,1
250038,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249985,0,1
1,250022,0,1
2,250034,0,1
3,250037,0,1
4,250038,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249985,,1
1,250022,,1
2,250034,,1
3,250037,,1
4,250038,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc."
1,Nov 4 2022 11:43AM,250069,10,ISDIN Corporation
4,Nov 4 2022 11:41AM,250068,19,"GUSA Granules USA, Inc."
5,Nov 4 2022 11:40AM,250065,10,ISDIN Corporation
53,Nov 4 2022 11:37AM,250067,20,"Emersa Waterbox, LLC"
54,Nov 4 2022 11:31AM,250066,19,"AdvaGen Pharma, Ltd"
55,Nov 4 2022 11:11AM,250063,10,"Senseonics, Incorporated"
56,Nov 4 2022 11:11AM,250062,10,"Senseonics, Incorporated"
58,Nov 4 2022 10:26AM,250056,15,"Alliance Pharma, Inc."
68,Nov 4 2022 10:18AM,250055,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc.",,1
1,Nov 4 2022 11:43AM,250069,10,ISDIN Corporation,,3
2,Nov 4 2022 11:41AM,250068,19,"GUSA Granules USA, Inc.",,1
3,Nov 4 2022 11:40AM,250065,10,ISDIN Corporation,,48
4,Nov 4 2022 11:37AM,250067,20,"Emersa Waterbox, LLC",,1
5,Nov 4 2022 11:31AM,250066,19,"AdvaGen Pharma, Ltd",,1
6,Nov 4 2022 11:11AM,250063,10,"Senseonics, Incorporated",,1
7,Nov 4 2022 11:11AM,250062,10,"Senseonics, Incorporated",,2
8,Nov 4 2022 10:26AM,250056,15,"Alliance Pharma, Inc.",10,
9,Nov 4 2022 10:18AM,250055,19,"AdvaGen Pharma, Ltd",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc.",1,
1,Nov 4 2022 11:43AM,250069,10,ISDIN Corporation,3,
2,Nov 4 2022 11:41AM,250068,19,"GUSA Granules USA, Inc.",1,
3,Nov 4 2022 11:40AM,250065,10,ISDIN Corporation,48,
4,Nov 4 2022 11:37AM,250067,20,"Emersa Waterbox, LLC",1,
5,Nov 4 2022 11:31AM,250066,19,"AdvaGen Pharma, Ltd",1,
6,Nov 4 2022 11:11AM,250063,10,"Senseonics, Incorporated",1,
7,Nov 4 2022 11:11AM,250062,10,"Senseonics, Incorporated",2,
8,Nov 4 2022 10:26AM,250056,15,"Alliance Pharma, Inc.",,10
9,Nov 4 2022 10:18AM,250055,19,"AdvaGen Pharma, Ltd",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc.",1,
1,Nov 4 2022 11:43AM,250069,10,ISDIN Corporation,3,
2,Nov 4 2022 11:41AM,250068,19,"GUSA Granules USA, Inc.",1,
3,Nov 4 2022 11:40AM,250065,10,ISDIN Corporation,48,
4,Nov 4 2022 11:37AM,250067,20,"Emersa Waterbox, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc.",1.0,NaN
1,Nov 4 2022 11:43AM,250069,10,ISDIN Corporation,3.0,NaN
2,Nov 4 2022 11:41AM,250068,19,"GUSA Granules USA, Inc.",1.0,NaN
3,Nov 4 2022 11:40AM,250065,10,ISDIN Corporation,48.0,NaN
4,Nov 4 2022 11:37AM,250067,20,"Emersa Waterbox, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc.",1.0,0.0
1,Nov 4 2022 11:43AM,250069,10,ISDIN Corporation,3.0,0.0
2,Nov 4 2022 11:41AM,250068,19,"GUSA Granules USA, Inc.",1.0,0.0
3,Nov 4 2022 11:40AM,250065,10,ISDIN Corporation,48.0,0.0
4,Nov 4 2022 11:37AM,250067,20,"Emersa Waterbox, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2000475,57.0,9.0
15,250056,0.0,10.0
16,500071,2.0,0.0
18,250022,1.0,0.0
19,1500308,6.0,0.0
20,250067,1.0,0.0
21,249985,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2000475,57.0,9.0
1,15,250056,0.0,10.0
2,16,500071,2.0,0.0
3,18,250022,1.0,0.0
4,19,1500308,6.0,0.0
5,20,250067,1.0,0.0
6,21,249985,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,57.0,9.0
1,15,0.0,10.0
2,16,2.0,0.0
3,18,1.0,0.0
4,19,6.0,0.0
5,20,1.0,0.0
6,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,15,Released,0.0
2,16,Released,2.0
3,18,Released,1.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Executing,9.0,10.0,0.0,0.0,0.0,0.0,0.0
Released,57.0,0.0,2.0,1.0,6.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Executing,9.0,10.0,0.0,0.0,0.0,0.0,0.0
1,Released,57.0,0.0,2.0,1.0,6.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Executing,9.0,10.0,0.0,0.0,0.0,0.0,0.0
1,Released,57.0,0.0,2.0,1.0,6.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()